In [94]:
import pandas as pd
import requests
import lxml.html as lh
%matplotlib inline

In [165]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(url)
doc=lh.fromstring(page.content)

In [166]:
elements=doc.xpath('//tr')#table is stored in <tr>..</tr>#

In [167]:
[len(x) for x in elements[:10]] #check the length of first 10 rows#

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [168]:
list=[]
i=0
for j in elements[0]:
    i+=1
    name=j.text_content()
    print ('%d:"%s"'%(i,name))
    list.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [169]:
for e in range(1,len(elements)):
    T=elements[e]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content()
        if i>0:
            try:
                data=int(data)
            except:
                pass
                
        list[i][1].append(data)
        i+=1

In [170]:
[len(x) for (title,x) in list]#the length of the columns#

[288, 288, 288]

In [171]:
dic={title:column for (title,column) in list}
df=pd.DataFrame(dic)

In [172]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [173]:
df['Neighbourhood\n'] = df['Neighbourhood\n'].map(lambda x: str(x)[:-1])

In [174]:
df.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace=True)

In [175]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [176]:
df=df[df.Borough!='Not assigned']#ignore the 'not assigned' in Borough#

In [177]:
df=df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [178]:
df=df.groupby(['Postcode','Borough'], as_index=False).agg(','.join)# merge the Neighbourhood with the same Postcode#

In [179]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [189]:
df[df['Neighbourhood'].str.contains("Not assigned")] #sign the same value with Borough if Neighbourhood is Not assigned#

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [192]:
df=df.replace('Not assigned','Queen"s Park')

In [194]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [195]:
df.shape

(103, 3)